In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/01 18:20:48 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.129 instead (on interface wlp4s0)
23/03/01 18:20:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 18:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/01 18:20:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/01 18:20:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/01 18:20:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/03/01 18:20:49 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
spark.version

'3.3.2'

In [ ]:
!ls -lh fhvhv_tripdata_2021-02.csv

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

df = df.repartition(24)

df.write.parquet('data/pq/fhvhv/2021/02/', compression=)

In [ ]:
df = spark.read.parquet('data/pq/fhvhv/2021/02/')

**Q3**: How many taxi trips were there on February 15?

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-02-15'") \
    .count()

In [ ]:
df.registerTempTable('fhvhv_2021_02')

In [ ]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_02
WHERE
    to_date(pickup_datetime) = '2021-02-15';
""").show()

**Q4**: Longest trip for each day

In [ ]:
df.columns

In [ ]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

In [ ]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [ ]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

In [ ]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

**Q6**: Most common locations pair

In [ ]:
df_zones = spark.read.parquet('zones')

In [ ]:
df_zones.columns

In [ ]:
df.columns

In [ ]:
df_zones.registerTempTable('zones')

In [ ]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_02 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()